In [1]:
countries = [
    ('CH', 'switzerland'),
    ('DE', 'germany'),
    ('FR', 'france'),
    ('IT', 'italy'),
    ('ES', 'spain')
]

In [2]:
country, country_name = countries[0]
country, country_name

('CH', 'switzerland')

In [3]:
import json

In [4]:
with open('../europe.json') as f:
    model = json.load(f)

In [5]:
{id: len(val) for id, val in model.items() if isinstance(val, dict)}

{'bus': 4097,
 'dcline': 0,
 'gen': 815,
 'branch': 8375,
 'storage': 0,
 'switch': 0,
 'shunt': 0,
 'load': 4097}

Delete all foreign buses:

In [6]:
for id in [id for id, bus in model['bus'].items() if bus['country'] != country]:
    del model['bus'][id]

Delete all foreign lines (including lines across the border):

In [7]:
for id in [id for id, line in model['branch'].items()
           if str(line['f_bus']) not in model['bus'] or str(line['t_bus']) not in model['bus']]:
    del model['branch'][id]

Check whether all nodes are still connected to the rest of the model:

In [8]:
bus_degree = {id: 0 for id in model['bus'].keys()}
for line in model['branch'].values():
    bus_degree[str(line['f_bus'])] += 1
    bus_degree[str(line['t_bus'])] += 1

In [9]:
disconnected_buses = {id for id, degree in bus_degree.items() if degree == 0}
disconnected_buses

{'5628'}

In [10]:
for id in disconnected_buses:
    del model['bus'][id]
    del bus_degree[id]

In [11]:
disconnected_pairs = {(id, (str(line['f_bus']), str(line['t_bus'])))
                      for id, line in model['branch'].items()
                      if bus_degree[str(line['f_bus'])] == 1 and bus_degree[str(line['t_bus'])] == 1}
disconnected_pairs

{('7974', ('5115', '5116'))}

In [12]:
for line_id, (bus_id_1, bus_id_2) in disconnected_pairs:
    del model['branch'][line_id]
    del model['bus'][bus_id_1]
    del model['bus'][bus_id_2]
    del bus_degree[bus_id_1]
    del bus_degree[bus_id_2]

Delete all foreign loads:

In [13]:
for id in [id for id, load in model['load'].items() if str(load['load_bus']) not in model['bus']]:
    del model['load'][id]

Delete all foreign gens:

In [14]:
for id in [id for id, gen in model['gen'].items() if str(gen['gen_bus']) not in model['bus']]:
    del model['gen'][id]

Set reference bus at the largest gen:

In [15]:
ref_buses = {id for id, bus in model['bus'].items() if bus['bus_type'] == 3}
ref_buses

set()

In [16]:
largest_gen = [id for id, gen in sorted(model['gen'].items(), key=lambda item: item[1]['pmax'])][-1]
model['gen'][largest_gen]

{'model': 2,
 'gen_bus': 5514,
 'pmax': 16.44,
 'mbase': 100,
 'vg': 1,
 'index': 914,
 'cost': [8000, 0],
 'gen_status': 1,
 'qmax': 8.22,
 'qmin': -8.22,
 'type': 'hydro_pure_storage',
 'pmin': 0,
 'ncost': 2,
 'country': 'CH',
 'entsoe_names': ['Usine de Bieudron', 'Usine de Nendaz'],
 'aggregated_type': 'hydro_storage',
 'pexp': 2.20000266221416}

In [17]:
if len(ref_buses) == 0:
    model['bus'][str(model['gen'][largest_gen]['gen_bus'])]['bus_type'] = 3

Change model name:

In [18]:
model['name'] = country_name

Remaining model:

In [19]:
{id: len(val) for id, val in model.items() if isinstance(val, dict)}

{'bus': 160,
 'dcline': 0,
 'gen': 36,
 'branch': 330,
 'storage': 0,
 'switch': 0,
 'shunt': 0,
 'load': 160}

Export:

In [20]:
model_json = json.dumps(model)
with open('../%s.json' % country_name, 'w') as f:
    f.write(model_json)